# CNF의 최대 가능도 추정(Maximum Likelihood Estimation)

이 노트북에서는 연속 정규화 흐름(Continuous Normalizing Flows)의 가능도(likelihood)를 계산하는 방법을 보여줍니다.

In [ ]:
import torch
import torch.nn as nn
import torch.distributions as D

import flow
import flow.utils

## 모델 정의하기

먼저, 간단한 모델을 정의해 보겠습니다. 이 모델은 입력 `x`에 대해 `t=0`에서 `t=1`까지의 흐름을 학습합니다.

이 모델은 `flow.Flow`를 상속하며, `flow.ContinuousNormalizingFlow` 클래스에 대한 동적(dynamics)으로 사용됩니다.

모델은 시간 `t`와 상태 `x`를 입력으로 받아 `dx/dt`를 출력하는 신경망을 정의합니다.

`training=True`로 설정하면, 이 모델은 정규화 항(regularization term)의 추정치를 계산하기 위해 추가적인 항을 계산합니다. 이 항들은 출력의 일부로 반환됩니다.

In [ ]:
class SimpleFlow(flow.Flow):
    def __init__(self, data_dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(data_dim + 1, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, data_dim)
        )
    
    def forward(self, t, x, *, training=False):
        # t를 x와 같은 차원으로 확장합니다.
        t_tiled = t.expand(x.shape[0], 1)
        
        # t와 x를 연결합니다.
        t_and_x = torch.cat([t_tiled, x], dim=1)
        
        # 신경망을 통과시킵니다.
        dx_dt = self.net(t_and_x)
        
        if not training:
            return dx_dt
        else:
            # 분산(divergence)의 추정치를 계산합니다.
            div = flow.utils.divergence_approx(dx_dt, x)
            
            # 운동 에너지(Kinetic Energy) 정규화 항을 계산합니다.
            kin_energy = torch.sum(dx_dt**2, dim=1)
            
            # 자카드(Jacobian) 정규화 항을 계산합니다.
            jac_reg = flow.utils.jacobian_frobenius_regularization_approx(dx_dt, x)
            
            return dx_dt, div, kin_energy, jac_reg

## CNF 정의하기

이제 `SimpleFlow`를 사용하여 `flow.ContinuousNormalizingFlow`의 인스턴스를 만들 수 있습니다.

이 클래스는 `flow.Flow` 인스턴스를 ODE 솔버(기본값: `dopri5`)와 함께 래핑(wrapping)하여, `t=0`에서 `t=1`까지의 흐름을 계산합니다.

`forward` 메소드는 `t=0`에서의 샘플을 `t=1`로 변환합니다. `inverse` 메소드는 그 반대로 변환합니다.

In [ ]:
data_dim = 2
hidden_dim = 64

cnf = flow.ContinuousNormalizingFlow(
    dynamics=SimpleFlow(data_dim, hidden_dim)
)

## 최대 가능도 추정

`cnf.loss(x)`를 호출하여 `x`의 음의 로그 가능도(negative log-likelihood)를 계산할 수 있습니다. 이는 `cnf.log_prob(x).mean()`과 동일합니다.

이 손실 함수는 세 가지 주요 구성 요소로 이루어집니다:

1. `log_p_z`: `t=1`에서 잠재 변수(latent variable) `z`의 로그 가능도
2. `delta_log_p`: `t=0`에서 `t=1`까지의 로그 밀도(log-density) 변화량
3. `reg_term`: 정규화 항

이 항들은 모두 **음의 로그 가능도**에 기여하도록 부호가 조정됩니다.

이 손실 함수를 최적화하면 모델이 데이터의 분포를 학습하게 됩니다.

In [ ]:
# 무작위 데이터 생성
x = torch.randn(128, data_dim)

# 손실 계산
loss = cnf.loss(x)

# 역전파 및 경사 하강법
loss.backward()

# 옵티마이저 업데이트 (여기에 옵티마이저 정의가 필요함)
# optimizer.step()

## 샘플링

모델을 학습시킨 후에는 `cnf.sample(n_samples)`를 사용하여 새로운 샘플을 생성할 수 있습니다.

이는 먼저 잠재 분포(latent distribution)에서 샘플링한 다음, `inverse` 메소드를 사용하여 `t=1`에서 `t=0`으로 변환하여 데이터 공간의 샘플을 생성합니다.

In [ ]:
samples = cnf.sample(100)

## 궤적(Trajectory) 계산하기

`cnf.trajectory(x)`를 사용하여 데이터 포인트 `x`가 `t=0`에서 `t=1`까지 어떻게 변환되는지 시각화할 수 있습니다.

마찬가지로, `cnf.inverse_trajectory(z)`를 사용하여 잠재 공간의 포인트 `z`가 `t=1`에서 `t=0`으로 어떻게 변환되는지 시각화할 수 있습니다.

In [ ]:
# 궤적 계산
trajectory = cnf.trajectory(x[0:1])

# 역궤적 계산
z = torch.randn(1, data_dim)
inverse_trajectory = cnf.inverse_trajectory(z)